# XAUUSD LSTM Colab AutoLoad
مدل LSTM برای پیش‌بینی قیمت طلا (XAUUSD)

In [ ]:
# این سلول برای نصب کتابخانه‌های لازم است
!pip install numpy pandas scikit-learn tensorflow matplotlib

In [ ]:
# این سلول برای بارگذاری فایل CSV از GitHub است
import pandas as pd
url = 'https://raw.githubusercontent.com/behicof/new-lsdm/main/xauusd_lstm_ready_clean.csv'
df = pd.read_csv(url)
df.head()